In [ ]:
# Rain and Snow

rain_df = df[df['class']==1]

X = rain_df[rain_df['num_hours']==0].drop(columns=cols_to_drop).values
y = rain_df[rain_df['num_hours']==0]['rainfall_hr'].values.reshape(-1,1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state=rs)

rain_df_test = df_test[df_test['class']==1]
X_test = rain_df_test[rain_df_test['num_hours']==0].drop(columns=cols_to_drop).values
y_test = rain_df_test[rain_df_test['num_hours']==0]['rainfall_hr'].values.reshape(-1,1)

#X_scaler = MinMaxScaler(feature_range=(0,1))
#y_scaler = MinMaxScaler(feature_range=(0,1))

#X_scaler = StandardScaler()
#y_scaler = StandardScaler()

#X_train = X_scaler.fit_transform(X_train)
#y_train = y_scaler.fit_transform(y_train)

#X_val = X_scaler.transform(X_val)
#y_val = y_scaler.transform(y_val)

#X_test = X_scaler.transform(X_test)
#y_test = y_scaler.transform(y_test)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_reg.score(X_val, y_val)

model = Sequential()
model.add(Dense(10, activation='relu', input_shape = (num_inputs,)))
model.add(Dense(10, activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))
predictions = model.predict(X_test)
r2_score(y_test, predictions)

i=2
print(predictions[i])
print(y_test[i])

df = pd.read_csv('basel_weather_7-1-09_to_7-1-19.csv', sep=';', header=10)
df = df.drop(columns=['Minute'])
column_names = ['year', 'month', 'day', 'hour', 'temperature', 'humidity', 'mslp', 'rainfall_hr', 'rainfall_lr', \
                'snowfall_hr', 'snowfall_lr', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']
df.columns = column_names
df = df.drop(columns=['snowfall_lr', 'rainfall_lr'])
df['hour'] = df['hour'].map(lambda x: (x + 2) % 24)
for i, row in df.iterrows():
      if row.hour == 0 or row.hour == 1:
        df.at[i,'day'] += 1

df['class_0hr'] = df.apply(lambda x: make_classes_extra(x.rainfall_hr, x.snowfall_hr), axis = 1)
for i in range(1, 13):
    col_name = 'class_' + str(i) + 'hr'
    shiftnum = i * -1
    df[col_name] = df['class_0hr'].shift(shiftnum)
    df = df.dropna()
    df[col_name] = df[col_name].astype(int)


df['class_12hr'].value_counts()


df_list = []
for i in range(0, 13):
    col_name = 'class_' + str(i) + 'hr'
    cols = ['year', 'month', 'day', 'hour', 'temperature', 'humidity', 'mslp', 'rainfall_hr', \
            'snowfall_hr', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
            'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
            'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust', col_name]
    tmp_df = df[cols]
    tmp_df = tmp_df.rename(index=str, columns={col_name: 'class'})
    df_class_0 = tmp_df[tmp_df['class'] == 0]
    df_class_1 = tmp_df[tmp_df['class'] == 1]
    df_class_2 = tmp_df[tmp_df['class'] == 2]
    df_class_3 = tmp_df[tmp_df['class'] == 3]
    df_class_4 = tmp_df[tmp_df['class'] == 4]
    
    count_class_0, count_class_2, count_class_1, count_class_4, count_class_3 = tmp_df['class'].value_counts()
    df_under_0 = df_class_0.sample(count_class_3)
    df_under_1 = df_class_1.sample(count_class_3)
    df_under_2 = df_class_2.sample(count_class_3)
    df_under_4 = df_class_4.sample(count_class_3)
    class_df = pd.concat([df_under_0, df_under_1, df_under_2, df_class_3, df_under_4], axis=0).reset_index(drop=True)
    class_df['num_hours'] = i
    df_list.append(class_df)

df_us_extra = pd.concat(df_list).reset_index(drop=True)
df_us_extra.head()

df_us_extra['class'].value_counts()

# Should be: 503 * 5 * 13
len(df_us_extra)

y_undersample = df_us_extra[df_us_extra['num_hours']==0]['class'].values
y_undersample_cat = to_categorical(df_us_extra[df_us_extra['num_hours']==0]['class'])
X_undersample = df_us_extra[df_us_extra['num_hours']==0][['temperature', 'humidity', 'mslp', \
                'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values

X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size=0.3, \
                                                        random_state=rs)

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
knn.score(X_test, y_test)

y_undersample = df_us_extra[df_us_extra['num_hours']==0]['class'].values.reshape(-1,)
y_undersample_cat = to_categorical(df_us_extra[df_us_extra['num_hours']==0]['class'])
X_undersample = df_us_extra[df_us_extra['num_hours']==0][['temperature', 'humidity', 'mslp', \
                'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values

X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample_cat, test_size=0.3, \
                                                        random_state=rs)

X_scaler = MinMaxScaler(feature_range=(0,1))

X_scaled_training = X_scaler.fit_transform(X_train)
X_scaled_testing = X_scaler.transform(X_test)

model = Sequential()
model.add(Dense(150, activation='relu', input_shape = (n_inputs,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', categorical_accuracy])
model.fit(X_scaled_training, y_train, epochs=100, validation_split=0.3)
predictions = model.predict(X_scaled_testing)

def categorize_rainfall(rainfall_hr):
    if rainfall_hr < 0.3:
        return 0
    else:
        return 1

rain_df = rain_df.drop(columns=['class'])
rain_df['class'] = rain_df['rainfall_hr'].apply(categorize_rainfall)

rain_df['class'].value_counts()

X = rain_df[rain_df['num_hours']==0].drop(columns=['class', 'num_hours', 'snowfall_hr', 'rainfall_hr', \
                                                   'year', 'month', 'day', 'hour']).values
y = rain_df[rain_df['num_hours']==0]['class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=rs)


knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
knn.score(X_test, y_test)

X = rain_df[rain_df['num_hours']==0].drop(columns=['class', 'num_hours', 'snowfall_hr', 'rainfall_hr', \
                                                   'year', 'month', 'day', 'hour']).values
y = to_categorical(rain_df[rain_df['num_hours']==0]['class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=rs)

model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_inputs,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', categorical_accuracy])
model.fit(X_train, y_train, epochs=100, validation_split=0.3)
predictions = model.predict(X_scaled_testing)











#X = df[df['num_hours']==0].drop(columns=cols_to_drop).values
#y = to_categorical(df[df['num_hours']==0]['class'])
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=rs)
#sm = SMOTE(random_state=rs)
#X_train, y_train = sm.fit_sample(X_train, y_train)

#X_scaler = MinMaxScaler(feature_range=(0,1))
#X_train = X_scaler.fit_transform(X_train)
#X_val = X_scaler.transform(X_val)

#X_test = df_test[df_test['num_hours']==0].drop(columns=cols_to_drop).values
#y_test = df_test[df_test['num_hours']==0]['class'].values.reshape(-1,)

X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(df, num_hours=0, do_std_scaler=True, \
                                                              do_pca=11, do_smote = True)

learning_rate = 0.001
training_epochs = 100
display_step = 5

num_inputs = X_train.shape[1]
num_outputs = 3

layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

tf.reset_default_graph()

with tf.variable_scope('inupt'):
    X = tf.placeholder(tf.float32, shape=(None, num_inputs))

with tf.variable_scope('layer1'):
    weights = tf.get_variable(name='weights1', shape=[num_inputs, layer_1_nodes], \
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_out = tf.nn.relu(tf.matmul(X, weights) + biases)

with tf.variable_scope('layer2'):
    weights = tf.get_variable(name='weights2', shape=[layer_1_nodes, layer_2_nodes], \
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_out = tf.nn.relu(tf.matmul(layer_1_out, weights) + biases)

with tf.variable_scope('layer3'):
    weights = tf.get_variable(name='weights3', shape=[layer_2_nodes, layer_3_nodes], \
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_out = tf.nn.relu(tf.matmul(layer_2_out, weights) + biases)

with tf.variable_scope('output'):
    weights = tf.get_variable(name='weights4', shape=[layer_3_nodes, num_outputs], \
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[num_outputs], initializer=tf.zeros_initializer())
    prediction = tf.nn.softmax(tf.matmul(layer_3_out, weights) + biases)

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 3))
    cost = tf.losses.softmax_cross_entropy(Y, prediction)

# define optimzier
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter('./logs/training', session.graph)
    testing_writer = tf.summary.FileWriter('./logs/testing', session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_train, Y: y_train})

         # Every few training steps, log our progress
        if epoch % display_step == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_train, Y:y_train})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_val, Y:y_val})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_train, Y: y_train})
    final_testing_cost = session.run(cost, feed_dict={X: X_val, Y: y_val})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))

    y_predicted = session.run(prediction, feed_dict={X: X_val})
    
    inputs = {
        'input': tf.saved_model.utils.build_tensor_info(X)
        }
    outputs = {
        'earnings': tf.saved_model.utils.build_tensor_info(prediction)
        }

i = 21
y_test_not_one_hot = np.argmax(y_val, axis=1)
if y_test_not_one_hot[i] == 0:
    weather = 'Clear'
elif y_test_not_one_hot[i] == 1:
    weather = 'Rain'
else:
    weather = 'Snow'

print('Sample Prediction from Neural Network:')
print('Percent Chance of Clear Weather: {}'.format(y_predicted[i][0]*100))
print('Percent Chance of Rain: {}'.format(y_predicted[i][1]*100))
print('Percent Chance of Snow: {}'.format(y_predicted[i][2]*100))
print('\n\nAcutal Weather: %s' %weather)

y_predicted_not_one_hot = np.argmax(y_predicted, axis=1) 
print('Neural Network Validation Set Accuracy: {}'\
      .format(np.sum((y_predicted_not_one_hot == y_test_not_one_hot) / len(y_test_not_one_hot))))